In [3]:
import sys
sys.path.append("..")

In [85]:
import nltk, re, string, time, os
import pandas as pd
import numpy as np
from textClassification import textAnalysis
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet as wn
from sklearn.metrics import accuracy_score
from sklearn import metrics 
from sklearn import cross_validation
english_stopwords = stopwords.words('english')

### Helper functions

In [3]:
# automate creation of csv for kaggle submission
def create_submission(filename):
    pd.DataFrame.to_csv(df_test[['Id','Category']], path_or_buf=filename, header=['Id','Category'], index=False)

In [106]:
# yell outloud 'process complete'
def finished():
    os.system("say 'process complete'")

In [79]:
# automate crossvalidation and printing of results
def crossvalidation(model, train_features, real_category):
    scores = cross_validation.cross_val_score(model, train_features, real_category, cv=5)
    print("Accuracy: %0.5f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

## Loading Data

In [11]:
df = pd.read_csv("newtrain.csv", low_memory=False)

In [12]:
df_test = pd.read_csv("newtest.csv", low_memory=False)

In [13]:
allText = ""
for i in df.Text:
    allText += i + '.'

In [14]:
f = open('allText.txt', 'w')

In [15]:
f.write(allText)
f.close()

## Testing functions from Text Analysis assignement on Yahoo questions text data

In [9]:
path = "allText.txt"

In [10]:
analysis = textAnalysis(path)

In [29]:
# tokenize sentences and words an run them through a POS tagger
sentences,tagged_sents = analysis.gen_corpus_prep()

# generate noun phrases and proper nouns
nps,nps_strings,fd_nps = analysis.chunker(sentences,tagged_sents)

# deduce synsets through lesk algorithm and calculate distance between synsets
final_list,hyponyms = analysis.lesk_comparison(nps,sentences)

# print final report
analysis.final_report(fd_nps,final_list,hyponyms)

Starting NP parsing.
Done with NP parsing.
Done with FD of NPs
Extracting nouns...
Done extracting nouns
Done running Lesk on nouns
Done creating FD on lemmas

Most frequent noun phrases:
---------------------------
.how do                 91   
br & gt                 27   
.what do                22   
my boyfriend            19   
a guy                   19   
my comput               16   
the world               15   
a girl                  14   
.whi do                 11   
.i am                   9    
.i want                 9    
my husband              9    


Most frequent lemmas from NPs and their hyponyms:
-------------------------------------------------
can             194      (containerful.n.01)
get             184      (return.n.11)
know            156      (knowing.n.01)
find            89       (act.n.02)
need            77       (psychological_feature.n.01)
want            70       (need.n.01)
people          58       (group.n.01)
someone         51       (organis

## Prepping data

In [16]:
cat_dict = dict()

In [17]:
cat_dict = {1:"Business&Finance",2:"Computers&Internet",3:"Entertainment&Music",4:"Family&Relationships",5:"Education&Reference",6:"Health",7:"Science&Mathematics"}

In [33]:
# creating feature for lengths of text and converting to pandas series
lengths = []
for i in df.Text.iteritems():
    lengths.append(len(i[1]))
lengths = pd.Series(lengths)

In [34]:
# appending lengths feature to dataframe
df['lengths'] = lengths

In [20]:
# aggregate all text
all_text = []
for i in df.Text:
    all_text.append(i)

In [21]:
# aggregate all text and tokenize it
tok_sent = []
for i in df.Text:
    tok_sent.append(nltk.word_tokenize(i))

In [22]:
# pos_tag every word
pos_sent = []
for i in tok_sent:
    pos_sent.append(nltk.pos_tag(i))

## Implementation and Testing of Strategies

### 1st Strategy: Count occurrence of every tag (17.6%)

In [35]:
# creating list with all tags from upenn_tagset
tagset = ['$', "''", '(', ')', ',', '--', '.', ':','CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']

In [36]:
# creating columns of zeroes for every possible tag
for i in tagset:
    df[i] = pd.Series(np.zeros(df.shape[0]))

In [37]:
# Tokenizing and tagging every question training set
tagged_questions = []
for i in df.Text:
    tagged_questions.append(nltk.pos_tag(nltk.word_tokenize(i)))

In [38]:
# Tokenizing and tagging every question for test set
tagged_questions_test = []
for i in df_test.Text:
    tagged_questions_test.append(nltk.pos_tag(nltk.word_tokenize(i)))

In [54]:
# This is taking WAAAY too long, expected runtime > 1 hour. Use sparce vector array instead
time_check = time.time()
for n,q in enumerate(tagged_questions[:10]): # for every tagged questios from 'df.Text'...
    for tag in tagset: # take the list of possible tags...
        for w in q: # for every tagged word in the question text
            if w[1] == tag: # check if the tag matches the one the word is tagged as
                df[tag][n] += 1 # if so, increase counter and update the dataframe
    if n+1 % 10 == 0:
        print("Done with first",n)
print(round((time.time() - time_check)/60, 2),"minutes")

0.27 minutes


/Users/nicolassoldi/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [57]:
finished()

In [98]:
# round up all tags in one list for train data
tags_only_train = []
for q in tagged_questions:
    temp = ""
    for w in q:
        temp += w[1]
        temp += " "
    tags_only_train.append(temp)

In [105]:
# round up all tags in one list for test data
tags_only_test = []
for q in tagged_questions_test:
    temp = ""
    for w in q:
        temp += w[1]
        temp += " "
    tags_only_test.append(temp)

In [107]:
finished()

In [125]:
vec = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=5, max_features=433)

In [126]:
# Train sparce vectorizing
arr_train_feature_sparse = vec.fit_transform(pd.Series(tags_only_train))
arr_train_feature = arr_train_feature_sparse.toarray()
arr_train_feature.shape

(2698, 433)

In [127]:
# Test sparce vectorizing
arr_test_feature_sparse = vec.fit_transform(pd.Series(tags_only_test))
arr_test_feature = arr_test_feature_sparse.toarray()
arr_test_feature.shape

(1874, 377)

In [128]:
''' Training logistic regression with features in sparce array (arr_train_feature) 
and labels (df.Category)'''
logreg = LogisticRegression()
logreg_model_train = logreg.fit(arr_train_feature, df.Category)

In [129]:
# predicting against train features in sparce array
logreg_predictions_train = logreg_model_train.predict(arr_train_feature)

In [84]:
# crossvalidating
crossvalidation(logreg, arr_train_feature, df.Category)

Accuracy: 0.34875 (+/- 0.04)


In [131]:
# automatically creates submission with whatever file name you pass as the argument
create_submission("submission_3.csv")

### Second strategy: simple vectorizer and logistic regression (20.9%)

In [81]:
# function to facilitate testing of variations in CountVecotrizer parameters (min_df, max_features, etc.)
def logPredict(vec):
    # Train-data sparce vectorizing
    arr_train_feature_sparse = vec.fit_transform(df.Text)
    arr_train_feature = arr_train_feature_sparse.toarray()
    print("arr_train_feature.shape:", arr_train_feature.shape)

    # Test-data sparce vectorizing
    arr_test_feature_sparse = vec.fit_transform(df_test.Text)
    arr_test_feature = arr_test_feature_sparse.toarray()
    print("arr_test_feature.shape:", arr_test_feature.shape)

    ''' Training logistic regression with features in sparce array (arr_train_feature) 
    and labels (df.Category)'''
    logreg = LogisticRegression()
    logreg_model_train = logreg.fit(arr_train_feature, df.Category)

    # predicting against train features in sparce array
    logreg_predictions_train = logreg_model_train.predict(arr_train_feature)

    # predicting against test features in sparce array
    logreg_predictions_test = logreg_model_train.predict(arr_test_feature)

    # print out crossvalidation scoring results
    crossvalidation(logreg, arr_train_feature, df.Category)
    
    return logreg_predictions_test

In [63]:
vec = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=5, max_features=1000)

In [82]:
logreg_predictions_test = logPredict(vec)

arr_train_feature.shape: (2698, 640)
arr_test_feature.shape: (1874, 640)
Accuracy: 0.34875 (+/- 0.04)


In [53]:
logreg_predictions_test

array([1, 1, 1, ..., 1, 1, 1])

In [194]:
# adding Category column to test set
df_test['Category'] = pd.Series(logreg_predictions_test)

In [195]:
# automatically creates submission with whatever file name you pass as the argument
create_submission("submission_5.csv")

### Third strategy: bigram, trigram and tetragram vectorizer and logistic regression (22.6%)

In [68]:
# notice the ngram_range parameter has "2, 4", so ngrams range 2 to 4
vec = CountVectorizer(ngram_range=(2, 4), token_pattern=r'\b\w+\b', min_df=5, max_features=640)

In [69]:
logreg_predictions_test = logPredict(vec)

arr_train_feature.shape: (2698, 640)
arr_test_feature.shape: (1874, 640)
Accuracy: 0.73904 (+/- 0.02)


In [73]:
# Train-data sparce vectorizing
arr_train_feature_sparse = vec.fit_transform(df.Text)
arr_train_feature = arr_train_feature_sparse.toarray()
print("arr_train_feature.shape:", arr_train_feature.shape)

# Test-data sparce vectorizing
arr_test_feature_sparse = vec.fit_transform(df_test.Text)
arr_test_feature = arr_test_feature_sparse.toarray()
print("arr_test_feature.shape:", arr_test_feature.shape)

''' Training logistic regression with features in sparce array (arr_train_feature) 
and labels (df.Category)'''
logreg = LogisticRegression()
logreg_model_train = logreg.fit(arr_train_feature, df.Category)

# predicting against train features in sparce array
logreg_predictions_train = logreg_model_train.predict(arr_train_feature)

arr_train_feature.shape: (2698, 640)
arr_test_feature.shape: (1874, 640)


In [75]:
crossvalidation(logreg, arr_train_feature, df.Category)

Accuracy: 0.34875 (+/- 0.04)


### Fourth strategy: same vectorizer but with Naive Bayes (%)

In [89]:
# function to facilitate testing of variations in CountVecotrizer parameters (min_df, max_features, etc.)
def NBPredict(vec):
    # Train-data sparce vectorizing
    arr_train_feature_sparse = vec.fit_transform(df.Text)
    arr_train_feature = arr_train_feature_sparse.toarray()
    print("arr_train_feature.shape:", arr_train_feature.shape)

    # Test-data sparce vectorizing
    arr_test_feature_sparse = vec.fit_transform(df_test.Text)
    arr_test_feature = arr_test_feature_sparse.toarray()
    print("arr_test_feature.shape:", arr_test_feature.shape)

    ''' Training logistic regression with features in sparce array (arr_train_feature) 
    and labels (df.Category)'''
    nb = MultinomialNB()
    nb_model_train = nb.fit(arr_train_feature, df.Category)

    # predicting against train features in sparce array
    nb_predictions_train = nb_model_train.predict(arr_train_feature)

    # predicting against test features in sparce array
    nb_predictions_test = nb_model_train.predict(arr_test_feature)

    # print out crossvalidation scoring results
    crossvalidation(nb, arr_train_feature, df.Category)
    
    return nb_predictions_test

In [87]:
# notice the ngram_range parameter has "2, 4", so ngrams range 2 to 4
vec = CountVectorizer(ngram_range=(2, 4), token_pattern=r'\b\w+\b', min_df=5, max_features=640)

In [90]:
nb_test_predictions = NBPredict(vec)

arr_train_feature.shape: (2698, 640)
arr_test_feature.shape: (1874, 640)
Accuracy: 0.34686 (+/- 0.05)
